$$\textbf{Dans ce fichier, nous allons essayer d'extraire les données des joueurs}\\
\textbf{de la Premier League de la saison 2020-2021.}\\
\textbf{Pour ce faire, nous allons utiliser les modules de Python suivants :}\\
$$
- **BeautifulSoup** pour extraire des informations à partir de pages Web. Elle prend en entrée un document HTML et le transforme en un arbre de syntaxe balisée.

- **Selenium** pour automatiser le navigateur et simuler l'interaction avec DOM de la page Web. Elle va nous simplifier la tache avec sa capacité d'interaction avec le navigateur pour ecouter les evenements et prevoir des callback une fois l'evenment déclenché

$$\textbf{Après avoir importé les modules nécessaires, nous allons utiliser}\\
\textbf{Selenium pour interagir avec le navigateur, écouter les événements}\\
\textbf{(de clic de bouton par exemple) et extraire les données qui nous intéressent.}\\
\textbf{Nous allons expliquer chaque ligne de code pour plus de transparence.}\\
\textbf{\LARGE 🎲}\\
$$

### **Installation des modules**

- **Installation de selenium:** *pip install selenium*
- **Installation de webdriver_manager:** *pip install webdriver_manager*
- **Installation de BeautifulSoup:** *pip install bs4*

### **Importations des modules necessaires pour faire le web scrapping**

- **Les modules de Selenium dont on aura besoin:**

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

- **Importation de BeautifulSoup, numpy et pandas**

In [3]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### **Lancement d'une instance de Chrome avec Selenium**

- Selelnium utilise un pilote pour fonctionner, chaque navigateur a son propre pilote. Dans notre cas, on utilise Chrome comme navigateur. On peut faire l'installation du pilote manuellement en allant sur le site officiel de Selenium, voir la pilote compatible avec la version de notre navigateur et l'installer. Comme on peut aller directement sur ce repositorty github https://github.com/SergeyPirogov/webdriver_manager et copier le code pour installer le pilote. Le code telecharge et installe le pilote directement.
*Utilisons webdriver de Selenium pour ouvrir chrome et le configurer pour qu'il fonctionne en mode silencieux*

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

- Ensuite on ouvre sur l'instance qu'on vient d'ouvrir

In [29]:
url = "https://www.premierleague.com/stats/top/players/goals"
driver.get(url) #charger la page Web à partir de l'URL.

# **Extraction des données**

Nous allons d'abord essayer de voir comment extraire les elements dont on a besoin pour chaque type de statistique.
Les données a recuperer:
- **Player**: Le nom du joueur
- **Club**: le club où joue le joueur
- **Nationality**: la nationalité du joueur
- **Stat**: la valeur du type de stat du joueur, par exemple si s'agit des buts, stat represente le nombre de buts marqués par le joeurs.
  
On a décidé de ne pas prendre la colonne **Rank** car un joueur peut prendre premier sur les butteurs et être 12eme sur les passeurs. Elles sera donc supprimé plustard!!

### **Essayons de voir d'abord comment extraire un a un ces statistiques**

Pour cet exemple, nous allons prendre les butteurs de la saison 2020-2021

- Apres avoir filtré l'année et le stat dont on a besoin, récupérons le contenu de la page filtrée avec la propriété *page_source* du navigateur pour récupérer le contenu HTML de la page Web, puis utiliser BeautifulSoup pour extraire les données nécessaires

In [38]:
soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web

- on utlise ensuite la methode *find_all* de BeautifulSoup pour trouver les balises qui nous interesse

In [39]:
page = soup.find_all("tr")
page

[<tr>
 <th class="rank" scope="col">Rank</th>
 <th scope="col">Player</th>
 <th class="hide-s" scope="col">Club</th>
 <th class="hide-s" scope="col">Nationality</th>
 <th class="mainStat text-centre" scope="col">Stat</th>
 <th class="order">
 <span class="reorderStatsButton">
 <span class="icn sort"></span>
 </span>
 </th>
 </tr>,
 <tr class=""> <td class="rank"><strong>1.</strong></td> <td> <a class="playerName" href="//www.premierleague.com/players/3960/player/overview"> <strong> Harry Kane</strong></a><strong> </strong></td> <td class="hide-s"> <a class="statNameSecondary" href="//www.premierleague.com/clubs/21/Tottenham-Hotspur/overview"> <span class="badge badge-image-container" data-size="25" data-widget="club-badge-image"> <img class="badge-image badge-image--25 js-badge-image" src="https://resources.premierleague.com/premierleague/badges/25/t6.png" srcset="https://resources.premierleague.com/premierleague/badges/25/t6.png, https://resources.premierleague.com/premierleague/badge

- Recuperation des noms des colonnes

In [40]:
columns = []
header_rows = page[0]
for column in header_rows.find_all('th'):
    columns.append(column.text)
columns

['Rank', 'Player', 'Club', 'Nationality', 'Stat', '\n\n\n\n']

In [41]:
del(columns[-1])
columns

['Rank', 'Player', 'Club', 'Nationality', 'Stat']

- Recuperation de Harry Kane

In [42]:
harry_kane = page[1]
harry_kane

<tr class=""> <td class="rank"><strong>1.</strong></td> <td> <a class="playerName" href="//www.premierleague.com/players/3960/player/overview"> <strong> Harry Kane</strong></a><strong> </strong></td> <td class="hide-s"> <a class="statNameSecondary" href="//www.premierleague.com/clubs/21/Tottenham-Hotspur/overview"> <span class="badge badge-image-container" data-size="25" data-widget="club-badge-image"> <img class="badge-image badge-image--25 js-badge-image" src="https://resources.premierleague.com/premierleague/badges/25/t6.png" srcset="https://resources.premierleague.com/premierleague/badges/25/t6.png, https://resources.premierleague.com/premierleague/badges/25/t6@x2.png 2x"/> </span>Tottenham Hotspur </a> </td> <td class="hide-s"> <div class="info"> <span class="flag GB-ENG"></span> <span class="playerCountry">England</span> </div> </td> <td class="mainStat text-centre">23</td> <td> </td> </tr>

In [43]:
harry_values = harry_kane.find_all("td")
harry_values

[<td class="rank"><strong>1.</strong></td>,
 <td> <a class="playerName" href="//www.premierleague.com/players/3960/player/overview"> <strong> Harry Kane</strong></a><strong> </strong></td>,
 <td class="hide-s"> <a class="statNameSecondary" href="//www.premierleague.com/clubs/21/Tottenham-Hotspur/overview"> <span class="badge badge-image-container" data-size="25" data-widget="club-badge-image"> <img class="badge-image badge-image--25 js-badge-image" src="https://resources.premierleague.com/premierleague/badges/25/t6.png" srcset="https://resources.premierleague.com/premierleague/badges/25/t6.png, https://resources.premierleague.com/premierleague/badges/25/t6@x2.png 2x"/> </span>Tottenham Hotspur </a> </td>,
 <td class="hide-s"> <div class="info"> <span class="flag GB-ENG"></span> <span class="playerCountry">England</span> </div> </td>,
 <td class="mainStat text-centre">23</td>,
 <td> </td>]

Le rang

In [44]:
rank = int(list(harry_values[0].find("strong").text)[0])
rank

1

Le nom

In [45]:
name = harry_values[1].find("strong").text.strip()
name

'Harry Kane'

Le club

In [46]:
club = harry_values[2].find("a").text.strip()
club

'Tottenham Hotspur'

La nationalité

In [47]:
nationality = harry_values[3].find_all("span")[1].text
nationality

'England'

Nombre de buts

In [48]:
goals = int(harry_values[4].text)
goals

23

In [49]:
Harry_Kane = [rank, name, club, nationality, goals]
Harry_Kane

[1, 'Harry Kane', 'Tottenham Hotspur', 'England', 23]

- Recupération des joueurs de la page avec une boucle

In [50]:
players = []
for i in range(1, len(page)):
    player = page[i].find_all("td")
    rank = int(list(player[0].find("strong").text)[0])
    name = player[1].find("strong").text.strip()
    if player[2].find("a") is None:
        club = np.NAN
    else:
        club = player[2].find("a").text.strip()
    nationality = player[3].find_all("span")[1].text
    goals = int(player[4].text)
    players.append([rank, name, club, nationality, goals])

players

[[1, 'Harry Kane', 'Tottenham Hotspur', 'England', 23],
 [2, 'Mohamed Salah', 'Liverpool', 'Egypt', 22],
 [3, 'Bruno Fernandes', 'Manchester United', 'Portugal', 18],
 [4, 'Patrick Bamford', 'Leeds United', 'England', 17],
 [4, 'Son Heung-Min', 'Tottenham Hotspur', 'South Korea', 17],
 [6, 'Dominic Calvert-Lewin', 'Everton', 'England', 16],
 [7, 'Jamie Vardy', 'Leicester City', 'England', 15],
 [8, 'Ollie Watkins', 'Aston Villa', 'England', 14],
 [9, 'Ilkay Gündogan', 'Manchester City', 'Germany', 13],
 [9, 'Alexandre Lacazette', nan, 'France', 13],
 [1, 'Kelechi Iheanacho', 'Leicester City', 'Nigeria', 12],
 [1, 'Danny Ings', nan, 'England', 12],
 [1, 'Callum Wilson', 'Newcastle United', 'England', 12],
 [1, 'Chris Wood', nan, 'New Zealand', 12],
 [1, 'Gareth Bale', nan, 'Wales', 11],
 [1, 'Matheus Pereira', nan, 'Brazil', 11],
 [1, 'Sadio Mané', nan, 'Senegal', 11],
 [1, 'Marcus Rashford', 'Manchester United', 'England', 11],
 [1, 'Wilfried Zaha', 'Crystal Palace', 'Cote D’Ivoire', 1

### **Utilisation de Selenium pour controler le click sur le bouton "suivant"**

Maintenant que nous savons commemt extraire chaque type de statistique, utilisons les evenments de click de boutton (le bouton suivant de la page) pour extraire les autres joueurs.
Avec Selenium, on peut manipuler les elements du DOM, par exemple cliquer sur un bouton a partir des fonctions disponibles sur selenium.

- Recuperation du bouton "suivant" a partir de sa classe

In [30]:
next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))

-- **WebDriverWait(driver, 10)**: Cela crée une instance de WebDriverWait avec un délai maximum de 10 secondes. Cela signifie que Selenium attendra jusqu'à 10 secondes pour que l'élément spécifié devienne cliquable avant de lancer une exception de temps d'attente dépassé.

-- **EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer"))**: C'est une condition d'attendre que l'élément devienne cliquable. Ici, l'élément est sélectionné en utilisant le *CSS_SELECTOR* et le nom de la classe du bouton "paginationNextContainer". Cette condition vérifie si l'élément est à la fois présent dans le DOM de la page et cliquable.

- Maintenant qu'on a recupéré le bouton, on peut utiliser la methode *click()* pour cliquer sur le bouton

Notons que le boutton doit être visible pour être cliqué (vous devez defiler jusqu'à ce que le boutton soit visible sur la page)

In [32]:
next_button.click()

Vous pouvez verifier que le boutton a été bien cliqué ✅

On pourra juste utiliser le même boucle qu'on avait fait precedemment pour recuper les données de la page

### **Créons maintenant une boucle qui clique sur chaque bouton et recupère les données sur un dataframe**

Pendant l'execution du block de code suivant, vous pouvez verifier sur le navigateur que les bouttons sont en train de se cliquer tout seul 😎🔥

In [11]:
goals = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        goal = int(player[4].text)
        goals.append([rank, name, club, nationality, goal])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
goals

[[1, 'Harry Kane', 'Tottenham Hotspur', 'England', 23],
 [2, 'Mohamed Salah', 'Liverpool', 'Egypt', 22],
 [3, 'Bruno Fernandes', 'Manchester United', 'Portugal', 18],
 [4, 'Patrick Bamford', 'Leeds United', 'England', 17],
 [4, 'Son Heung-Min', 'Tottenham Hotspur', 'South Korea', 17],
 [6, 'Dominic Calvert-Lewin', 'Everton', 'England', 16],
 [7, 'Jamie Vardy', 'Leicester City', 'England', 15],
 [8, 'Ollie Watkins', 'Aston Villa', 'England', 14],
 [9, 'Ilkay Gündogan', 'Manchester City', 'Germany', 13],
 [9, 'Alexandre Lacazette', nan, 'France', 13],
 [1, 'Kelechi Iheanacho', 'Leicester City', 'Nigeria', 12],
 [1, 'Danny Ings', nan, 'England', 12],
 [1, 'Callum Wilson', 'Newcastle United', 'England', 12],
 [1, 'Chris Wood', nan, 'New Zealand', 12],
 [1, 'Gareth Bale', nan, 'Wales', 11],
 [1, 'Matheus Pereira', nan, 'Brazil', 11],
 [1, 'Sadio Mané', nan, 'Senegal', 11],
 [1, 'Marcus Rashford', 'Manchester United', 'England', 11],
 [1, 'Wilfried Zaha', 'Crystal Palace', 'Cote D’Ivoire', 1

## **Les buteurs**

- Boucle qui recupère les données

In [33]:
goals = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        goal = int(player[4].text)
        goals.append([rank, name, club, nationality, goal])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
goals

[[1, 'Harry Kane', 'Tottenham Hotspur', 'England', 23],
 [2, 'Mohamed Salah', 'Liverpool', 'Egypt', 22],
 [3, 'Bruno Fernandes', 'Manchester United', 'Portugal', 18],
 [4, 'Patrick Bamford', 'Leeds United', 'England', 17],
 [4, 'Son Heung-Min', 'Tottenham Hotspur', 'South Korea', 17],
 [6, 'Dominic Calvert-Lewin', 'Everton', 'England', 16],
 [7, 'Jamie Vardy', 'Leicester City', 'England', 15],
 [8, 'Ollie Watkins', 'Aston Villa', 'England', 14],
 [9, 'Ilkay Gündogan', 'Manchester City', 'Germany', 13],
 [9, 'Alexandre Lacazette', nan, 'France', 13],
 [1, 'Kelechi Iheanacho', 'Leicester City', 'Nigeria', 12],
 [1, 'Danny Ings', nan, 'England', 12],
 [1, 'Callum Wilson', 'Newcastle United', 'England', 12],
 [1, 'Chris Wood', nan, 'New Zealand', 12],
 [1, 'Gareth Bale', nan, 'Wales', 11],
 [1, 'Matheus Pereira', nan, 'Brazil', 11],
 [1, 'Sadio Mané', nan, 'Senegal', 11],
 [1, 'Marcus Rashford', 'Manchester United', 'England', 11],
 [1, 'Wilfried Zaha', 'Crystal Palace', 'Cote D’Ivoire', 1

- Convertion en Dataframe

In [34]:
goals = pd.DataFrame(goals, columns=columns)
goals.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [35]:
goals.to_csv('goals.csv', index=False)

## **Les passeurs decisives**

- Boucle qui recupère les données

In [28]:
assists = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        assist = int(player[4].text)
        assists.append([rank, name, club, nationality, assist])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
assists

[[1, 'Harry Kane', 'Tottenham Hotspur', 'England', 14],
 [2, 'Bruno Fernandes', 'Manchester United', 'Portugal', 12],
 [2, 'Kevin De Bruyne', 'Manchester City', 'Belgium', 12],
 [4, 'Jack Grealish', nan, 'England', 10],
 [4, 'Son Heung-Min', 'Tottenham Hotspur', 'South Korea', 10],
 [6, 'Raphinha', nan, 'Brazil', 9],
 [6, 'Marcus Rashford', 'Manchester United', 'England', 9],
 [6, 'Jamie Vardy', 'Leicester City', 'England', 9],
 [9, 'Aaron Cresswell', 'West Ham United', 'England', 8],
 [9, 'Pascal Groß', 'Brighton & Hove Albion', 'Germany', 8],
 [9, 'Jack Harrison', nan, 'England', 8],
 [9, 'Timo Werner', nan, 'Germany', 8],
 [1, 'Trent Alexander-Arnold', 'Liverpool', 'England', 7],
 [1, 'Patrick Bamford', 'Leeds United', 'England', 7],
 [1, 'Roberto Firmino', 'Liverpool', 'Brazil', 7],
 [1, 'Vladimír Coufal', 'West Ham United', '[object Object]', 7],
 [1, 'Lucas Digne', nan, 'France', 7],
 [1, 'Sadio Mané', nan, 'Senegal', 7],
 [1, 'Andrew Robertson', 'Liverpool', 'Scotland', 7],
 [1,

- Convertion en Dataframe

In [ ]:
assists = pd.DataFrame(assists, columns=columns)
assists.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
assists.to_csv('assists.csv', index=False)

## **Les clean-sheets des gardiens**

- Boucle qui recupère les données

In [39]:
clean_sheets = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        clean_sheet = int(player[4].text)
        clean_sheets.append([rank, name, club, nationality, clean_sheet])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
    
clean_sheets

[[1, 'Ederson', 'Manchester City', 'Brazil', 19],
 [2, 'Édouard Mendy', 'Chelsea', 'Senegal', 16],
 [3, 'Emiliano Martínez', 'Aston Villa', 'Argentina', 15],
 [4, 'Hugo Lloris', 'Tottenham Hotspur', 'France', 12],
 [5, 'Bernd Leno', nan, 'Germany', 11],
 [5, 'Illan Meslier', 'Leeds United', 'France', 11],
 [5, 'Nick Pope', nan, 'England', 11],
 [5, 'Kasper Schmeichel', nan, 'Denmark', 11],
 [9, 'Alisson', 'Liverpool', 'Brazil', 10],
 [9, 'Rui Patrício', nan, 'Portugal', 10],
 [9, 'Lukasz Fabianski', 'West Ham United', 'Poland', 10],
 [9, 'Jordan Pickford', 'Everton', 'England', 10],
 [9, 'Robert Sánchez', 'Brighton & Hove Albion', 'Spain', 10],
 [1, 'Alphonse Aréola', nan, 'France', 9],
 [1, 'David de Gea', 'Manchester United', 'Spain', 9],
 [1, 'Vicente Guaita', 'Crystal Palace', 'Spain', 8],
 [1, 'Alex McCarthy', 'Southampton', 'England', 7],
 [1, 'Sam Johnstone', nan, 'England', 6],
 [1, 'Aaron Ramsdale', nan, 'England', 5],
 [2, 'Karl Darlow', nan, 'England', 4]]

- Convertion en Dataframe

In [ ]:
clean_sheets = pd.DataFrame(clean_sheets, columns=columns)
clean_sheets.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
clean_sheets.to_csv('clean_sheets.csv', index=False)

## **Le nombre d'apparances des joueurs**

- Boucle qui recupère les données

In [ ]:
appearances = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        appearance = int(player[4].text)
        appearances.append([rank, name, club, nationality, appearance])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
appearances

- Convertion en Dataframe

In [ ]:
appearances = pd.DataFrame(appearances, columns=columns)
appearances.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
appearances.to_csv('appearances.csv', index=False)

## **Les nombre de minutes jouées**

- Boucle qui recupère les données

In [ ]:
minutes = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        minute = int("".join(player[4].text.split(",")))
        minutes.append([rank, name, club, nationality, minute])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
minutes

- Convertion en Dataframe

In [ ]:
minutes = pd.DataFrame(minutes, columns=columns)
minutes.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
minutes.to_csv('minutes.csv', index=False)

## **Le nombre de carton jaunes**

- Boucle qui recupère les données

In [ ]:
yellow_cards = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        yellow_card = int(player[4].text)
        yellow_cards.append([rank, name, club, nationality, yellow_card])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
yellow_cards

- Convertion en Dataframe

In [ ]:
yellow_cards = pd.DataFrame(yellow_cards, columns=columns)
yellow_cards.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
yellow_cards.to_csv('yellow_cards.csv', index=False)

## **Le nombre de cartons rouges**

- Boucle qui recupère les données

In [ ]:
red_cards = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        red_card = int(player[4].text)
        red_cards.append([rank, name, club, nationality, red_card])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
red_cards

- Convertion en Dataframe

In [ ]:
red_cards = pd.DataFrame(red_cards, columns=columns)
red_cards.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
red_cards.to_csv('red_cards.csv', index=False)

## **Le nombre de touchés de balles reussis**

- Boucle qui recupère les données

In [ ]:
touches = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        touch = int("".join(player[4].text.split(",")))
        touches.append([rank, name, club, nationality, touch])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
touches

- Convertion en Dataframe

In [ ]:
touches = pd.DataFrame(touches, columns=columns)
touches.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
touches.to_csv('touches.csv', index=False)

## **Le nombre de tirs**

- Boucle qui recupère les données

In [ ]:
shots = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        shot = int(player[4].text)
        shots.append([rank, name, club, nationality, shot])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
shots

- Convertion en Dataframe

In [ ]:
shots = pd.DataFrame(shots, columns=columns)
shots.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
shots.to_csv('shots.csv', index=False)

## **Le nombre de tirs sur le poteau ou la barre transvesale**

- Boucle qui recupère les données

In [ ]:
hit_woodworks = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        hit_woodwork = int(player[4].text)
        hit_woodworks.append([rank, name, club, nationality, hit_woodwork])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
hit_woodworks

- Convertion en Dataframe

In [ ]:
hit_woodworks = pd.DataFrame(assists, columns=columns)
hit_woodworks.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
hit_woodworks.to_csv('hit_woodworks.csv', index=False)

## **Le nombre de grosses occasions ratés**

- Boucle qui recupère les données

In [ ]:
big_chances_missed = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        big_chance_missed = int(player[4].text)
        big_chances_missed.append([rank, name, club, nationality, big_chance_missed])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
big_chances_missed

- Convertion en Dataframe

In [ ]:
big_chances_missed = pd.DataFrame(big_chances_missed, columns=columns)
big_chances_missed.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
big_chances_missed.to_csv('big_chances_missed.csv', index=False)

## **Le nombre d'hors-jeu**

- Boucle qui recupère les données

In [ ]:
offsides = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        offside = int(player[4].text)
        offsides.append([rank, name, club, nationality, offside])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
offsides

- Convertion en Dataframe

In [ ]:
offsides = pd.DataFrame(offsides, columns=columns)
offsides.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
offsides.to_csv('offsides.csv', index=False)

## **Le nombre de tacles**

- Boucle qui recupère les données

In [ ]:
tackles = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        tackle = int(player[4].text)
        tackles.append([rank, name, club, nationality, tackle])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
tackles

- Convertion en Dataframe

In [ ]:
tackles = pd.DataFrame(tackles, columns=columns)
tackles.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
tackles.to_csv('tackles.csv', index=False)

## **Le nombre de fautes commis**

- Boucle qui recupère les données

In [ ]:
fouls = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        foul = int(player[4].text)
        fouls.append([rank, name, club, nationality, foul])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
fouls

- Convertion en Dataframe

In [ ]:
fouls = pd.DataFrame(fouls, columns=columns)
fouls.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
fouls.to_csv('fouls.csv', index=False)

## **Le nombre de pertes de balle**

- Boucle qui recupère les données

In [ ]:
dispossess = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        lost = int(player[4].text)
        dispossess.append([rank, name, club, nationality, lost])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
dispossess

- Convertion en Dataframe

In [ ]:
dispossess = pd.DataFrame(dispossess, columns=columns)
dispossess.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
dispossess.to_csv('dispossess.csv', index=False)

## **Le nombre de buts contre son camp**

- Boucle qui recupère les données

In [ ]:
own_goals = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        own_goal = int(player[4].text)
        own_goals.append([rank, name, club, nationality, own_goal])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
own_goals

- Convertion en Dataframe

In [ ]:
own_goals = pd.DataFrame(own_goals, columns=columns)
own_goals.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
own_goals.to_csv('own_goals.csv', index=False)

## **Le nombre de degagements hors des cages (sauvetages)**

- Boucle qui recupère les données

In [ ]:
clearances = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        clearance = int(player[4].text)
        clearances.append([rank, name, club, nationality, clearance])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
clearances

- Convertion en Dataframe

In [ ]:
clearances = pd.DataFrame(clearances, columns=columns)
clearances.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
clearances.to_csv('clearances.csv', index=False)

## **Le nombre de sauvetages juste avant que la balle ne franchis la ligne de but**

- Boucle qui recupère les données

In [ ]:
off_lines = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        off_line = int(player[4].text)
        off_lines.append([rank, name, club, nationality, off_line])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
off_lines

- Convertion en Dataframe

In [ ]:
off_lines = pd.DataFrame(off_lines, columns=columns)
off_lines.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
off_lines.to_csv('off_lines.csv', index=False)

## **Le nombre de sauvetages de balle des gardiens**

- Boucle qui recupère les données

In [ ]:
saves = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        save = int(player[4].text)
        saves.append([rank, name, club, nationality, save])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
saves

- Convertion en Dataframe

In [ ]:
saves = pd.DataFrame(saves, columns=columns)
saves.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
saves.to_csv('saves.csv', index=False)

## **Le nombre de penalties sauvés par les gardiens de but**

- Boucle qui recupère les données

In [ ]:
penalty_saves = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        penalty_save = int(player[4].text)
        penalty_saves.append([rank, name, club, nationality, penalty_save])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
penalty_saves

- Convertion en Dataframe

In [ ]:
penalty_saves = pd.DataFrame(penalty_saves, columns=columns)
penalty_saves.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
penalty_saves.to_csv('big_chances_missed.csv', index=False)

## **Le nombre d'arrêts en attrappant le ballon des gardiens**

- Boucle qui recupère les données

In [ ]:
high_claims = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        high_claim = int(player[4].text)
        high_claims.append([rank, name, club, nationality, high_claim])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
high_claims

- Convertion en Dataframe

In [ ]:
high_claims = pd.DataFrame(high_claims, columns=columns)
high_claims.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
high_claims.to_csv('high_claims.csv', index=False)

## **Le nombre d'arrêts en boxant le ballon**

- Boucle qui recupère les données

In [ ]:
punches = []

while True:
    #on recupère la page sur une variable
    soup = BeautifulSoup(driver.page_source, "html.parser") #récupérer le contenu HTML de la page Web
    page = soup.find_all("tr")

    # on recupere les joeurs de la page en le mettant sur la liste goals
    for i in range(1, len(page)):
        player = page[i].find_all("td")
        rank = int(list(player[0].find("strong").text)[0])
        name = player[1].find("strong").text.strip()
        if player[2].find("a") is None:
            club = np.NAN
        else:
            club = player[2].find("a").text.strip()
        nationality = player[3].find_all("span")[1].text
        punch = int(player[4].text)
        punches.append([rank, name, club, nationality, punch])
    
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationNextContainer")))
    next_button.click()

    if "inactive" in next_button.get_attribute("class"):
        break
punches

- Convertion en Dataframe

In [ ]:
punches = pd.DataFrame(punches, columns=columns)
punches.head()

,Rank,Player,Club,Nationality,Stat
0,1,Harry Kane,Tottenham Hotspur,England,23
1,2,Mohamed Salah,Liverpool,Egypt,22
2,3,Bruno Fernandes,Manchester United,Portugal,18
3,4,Patrick Bamford,Leeds United,England,17
4,4,Son Heung-Min,Tottenham Hotspur,South Korea,17


- Sauvegarde dans un fichier csv

In [ ]:
punches.to_csv('punches.csv', index=False)

In [27]:
driver.quit()